In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import os

#local package
import segUtils

In [ ]:
# Load images of the same plant species
# Create the imgs directory for storing the segmented pollen images. 
# Fill it in with the images.
plant_names = ['cichorei', 'keizerskaars', 'phacelia', 'wondklaver']

files = os.listdir('.')
image_size = 600

if not os.path.exists('./imgs/'):
    os.makedirs('./imgs/')
    
for plant in plant_names: 
    i=1
    for filename in files:
        if plant in filename and 'jpg' in filename:
            image = plt.imread('./'+filename)
            extracted_images = utils.extractDarkGrains(image, image_size)
            for im in extracted_images:
                mpl.image.imsave("imgs/"+plant+"_" + str(i) + '.png', im)
                i = i + 1